# Code computes $\Gamma$ matrix for RE for a specific example.

### Setup
We take a simple two site Hamiltonian connected to bosonic bath, and compute the Redfield equation for it. Let us denote by $\hat{H}$ the Hamiltonian of the full set-up, including the system and all the baths,
$\hat{H}=\hat{H}_S + \epsilon \hat{H}_{SB} + \hat{H}_B$.

Let the system consist of a two site XXZ spin chain given by $$
\hat{H}_S =  \frac{\omega_0}{2} (\hat{\sigma}_z^{(1)} +\hat{\sigma}_z^{(2)}) -   g(\hat{\sigma}_x^{(1)} \hat{\sigma}_x^{(2)} + \hat{\sigma}_y^{(1)} \hat{\sigma}_y^{(2)} + \Delta \hat{\sigma}_z^{(1)} \hat{\sigma}_z^{(2)} )$$
 $$
 \hat{H}_{SB}  =\sum_{r=1}^\infty  (\kappa_{ r} \hat{B}^{ \dagger}_r \hat{\sigma}^{(1)}_{-} + \kappa_{r}^* \hat{B}_r \hat{\sigma}^{(1)}_+)$$
 $$
 \hat{H}_B= \sum_{r=1}^\infty \Omega_r \hat{B}^{ \dagger}_r \hat{B}_r $$


The RE for this equation can be computed to be $$
\frac{\partial \rho}{\partial t}  =i [\rho, \hat{H}_S]  - \epsilon ^2   \sum_{\alpha, \gamma=1}^{2^N}  \Big\{   \left[ \rho  \ket{E_\alpha} \bra{E_\alpha} \hat{\sigma}^1_-  \ket{E_\gamma} \bra{E_\gamma} ,  \hat{\sigma}^l_+   \right]   C(\alpha,\gamma) +\left[\hat{\sigma}^1_+  ,  \ket{E_\alpha} \bra{E_\alpha} \hat{\sigma}^l_-  \ket{E_\gamma}\bra{E_\gamma} \rho \right]  D(\alpha,\gamma) +\text{H.c}  \Big\}$$






where
$$
C(\alpha,\gamma) = \frac{\mathfrak{J}(E_{\gamma \alpha}) n(E_{\gamma \alpha})}{2 } - i \mathcal{P} \int_{0}^{\infty} d \omega \frac{\mathfrak{J}(\omega) n(\omega)}{\omega-E_{\gamma \alpha}},  \\
D(\alpha,\gamma) = \frac{ e^{\beta(E_{\gamma \alpha}-\mu_{1})} \mathfrak{J}(E_{\gamma \alpha}) n(E_{\gamma \alpha})}{2 }- i \mathcal{P} \int_{0}^{\infty} d \omega \frac{e^{\beta(\omega-\mu)} \mathfrak{J}(\omega) n(\omega)}{\omega-E_{\gamma \alpha}},
$$

Here $\ket{E_\alpha}$ and $\ket{E_\gamma}$ are eigenkets of the system hamiltonian and, with eigenenergies $E_\alpha$ and $E_\gamma$, where $E_{\gamma \alpha} = E_\gamma-E_\alpha$. The dynamics of the system are governed by the bath spectral functions, defined as $\mathfrak{J}(\omega)=  \sum_k 2 \pi \left| \kappa_{ k} \right|^2 \delta (\omega-\Omega_k) $ and the Bose distribution, $n(\omega)=[e^{\beta_(\omega-\mu)}- 1]^{-1}$, 

We have shown, any generic RE is given by $$\frac{\partial {\rho}}{\partial t}=i[ \rho(t),H_S]  + \epsilon^2  \sum_{l}  [S^\dagger_l , S_l^{(1)} \rho(t) ] - [S^\dagger_l , \rho(t) S_l^{(2)}] + \text{H.c}. $$ 

Comparing the two equations, we can see that there is only value of $l$ since there is only one bath, and 
$$ 
S^\dagger_l = \sigma_+^{(1)}, \quad S_l = \sigma_-^{(1)} 
$$
$$
 \quad S_l^{(1)} =  \ket{E_\alpha} \bra{E_\alpha} \hat{\sigma}^l_-  \ket{E_\gamma}\bra{E_\gamma}   D(\alpha,\gamma), \quad S_l^{(2)} = \ket{E_\alpha} \bra{E_\alpha} \hat{\sigma}^1_-  \ket{E_\gamma} \bra{E_\gamma}   C(\alpha,\gamma) 
$$

Therefore the above operators can be constructed numerically. Then we must prepare an appropriate orthonormal basis $F_\alpha$ for the operators, and then expand the above operators in that basis, giving
$$
S_l=\sum_{\alpha=1}^{d^2} a_{l \alpha} F_\alpha   (a_{l \alpha}=0,  \forall  d_L^2 \leq \alpha \leq d^2 -1 )  $$ 
$$
S^\dagger_l=\sum_{\alpha=1}^{d^2} a^\prime_{l \alpha} F_\alpha   (a^\prime_{l \alpha}=0,  \forall  d_L^2 \leq \alpha \leq d^2 -1 ) $$
$$
S^{(1)}_l=\sum_{\alpha=1}^{d^2} b_{l \alpha} F_\alpha, \quad   S^{(1) \dagger}_l=\sum_{\alpha=1}^{d^2} b^\prime_{l \alpha} F_\alpha  $$
$$
S^{(2)}_l=\sum_{\alpha=1}^{d^2} c_{l \alpha} F_\alpha , \quad  S^{(2) \dagger }_l=\sum_{\alpha=1}^{d^2} c^\prime_{l \alpha} F_\alpha 
$$

Then, $\Gamma$ matrix will given interms of the the above basis and coefficients.

## Code starts

In [1]:
from qutip import *
import numpy as np
from scipy import integrate
from helper_code_qutip import * 
import pandas as pd

In [2]:

b = 500 #upper bound on integration
limit_value = 700
N = 2
g= 0.1
w0 = 1

w0list= [1,1]
glist = [g]

tb = 0.01
epsilon = 0.1

beta=1
mu=-0.5
    
delta=1


In [3]:

c_1 = create_sm(N,1) # minus operator on first spin.

H_S = create_hamiltonian(w0list,glist,delta,N) 

eigenergies,eigstates=H_S.eigenstates()
number = len(eigenergies)


In [4]:
## We now start computing the integrals needed to compute the C and D coeefficients

integral11 = np.empty((number,number),dtype = np.cdouble) # stores J*N integral
integral12 = np.empty((number,number),dtype = np.cdouble) # stores J integral

for i in range(number):
    for k in range(number):
        freq = eigenergies[k]-eigenergies[i]

        if( np.absolute(freq) >= 1/10**10):
            integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta,mu,1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
            integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #left bath done
                
        if (np.absolute(freq)<=1/10**10):
            integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu,1),limit=limit_value)[0]
            integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,1),limit=limit_value)[0]
                
            
            
   

In [5]:
 
constantD=np.empty((number,number),dtype=np.cdouble)
constantC=np.empty((number,number),dtype=np.cdouble)

for i in range(number):
    for k in range(number):
        constantD[i,k]=integral12[i,k]+integral11[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,1)+func1(eigenergies[k]-eigenergies[i],tb,beta,mu,1))    # this is nbar+1
        constantC[i,k]=integral11[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta,mu,1)   # this is nbar
            


In [6]:
# Construct S_1

S_1 = 0
S_2 = 0

for i in range(number):
    for k in range(number):
       
        S_1 = S_1 + constantD[i,k]*eigstates[i].dag()*c_1*eigstates[k]*eigstates[i]*eigstates[k].dag()
        S_2 = S_2 + constantC[i,k]*eigstates[i].dag()*c_1*eigstates[k]*eigstates[i]*eigstates[k].dag()


   
        

## Orthonormal basis
Now that we are done with constructing all our operators, we will have to construct our orthonormal basis for $f_i$, $g_i$. 

In [7]:
dim1 = 2
dim2 = 2 #dimensions of the two parts of the system
d=dim1*dim2

list1 = create_basis_startingwith_identity(dim1)
list2 = create_basis_startingwith_identity(dim2)

ortho_list1 = GramSchmidt(list1)
ortho_list2 = GramSchmidt(list2)


In [8]:

basis_list = []



for i in range(dim1**2-1):
    oper = tensor(ortho_list1[i],ortho_list2[-1])
    basis_list.append(oper)

for index1 in range(dim1**2):
    for index2 in range((dim2**2)-1):

        oper = tensor(ortho_list1[index1],ortho_list2[index2])
        basis_list.append(oper)

if len(basis_list) != (d)**2-1:
    print("basis is of wrong length", len(basis_list))




We now have basis_list with our correct basis. Next we coompute the coeefficients. The relevant formulae are

$$
S^\dagger_l = \sigma_+^{(1)}, \quad S_l = \sigma_-^{(1)} 
$$
$$ 
\quad S_l^{(1)} =  \ket{E_\alpha} \bra{E_\alpha} \hat{\sigma}^l_-  \ket{E_\gamma}\bra{E_\gamma}   D(\alpha,\gamma), \quad S_l^{(2)} = \ket{E_\alpha} \bra{E_\alpha} \hat{\sigma}^1_-  \ket{E_\gamma} \bra{E_\gamma}   C(\alpha,\gamma) 
$$

$$
S_l=\sum_{\alpha=1}^{d^2} a_{l \alpha} F_\alpha   (a_{l \alpha}=0,  \forall  d_L^2 \leq \alpha \leq d^2 -1 )  
$$
$$
S^\dagger_l=\sum_{\alpha=1}^{d^2} a^\prime_{l \alpha} F_\alpha   (a^\prime_{l \alpha}=0,  \forall  d_L^2 \leq \alpha \leq d^2 -1 ) $$
$$
S^{(1)}_l=\sum_{\alpha=1}^{d^2} b_{l \alpha} F_\alpha, \quad   S^{(1) \dagger}_l=\sum_{\alpha=1}^{d^2} b^\prime_{l \alpha} F_\alpha  $$
$$
S^{(2)}_l=\sum_{\alpha=1}^{d^2} c_{l \alpha} F_\alpha , \quad  S^{(2) \dagger }_l=\sum_{\alpha=1}^{d^2} c^\prime_{l \alpha} F_\alpha 
$$

In [9]:
# computing a
a_coeff = np.zeros((d*d),dtype = np.cdouble)
aprime_coeff = np.zeros((d*d),dtype = np.cdouble)
b_coeff = np.zeros((d*d),dtype = np.cdouble)
bprime_coeff = np.zeros((d*d),dtype = np.cdouble)
c_coeff = np.zeros((d*d),dtype = np.cdouble)
cprime_coeff = np.zeros((d*d),dtype = np.cdouble)

for index in range(d*2):
    a_coeff[index] = hilbert_schmidt_innerproduct(basis_list[index],c_1.dag())
    aprime_coeff[index] = hilbert_schmidt_innerproduct(basis_list[index],c_1)

    b_coeff[index] = hilbert_schmidt_innerproduct(basis_list[index],S_1)
    bprime_coeff[index] = hilbert_schmidt_innerproduct(basis_list[index],S_1.dag())

    c_coeff[index] = hilbert_schmidt_innerproduct(basis_list[index],S_2)
    cprime_coeff[index] = hilbert_schmidt_innerproduct(basis_list[index],S_2.dag())

Constructing the $\gamma$ matrix. (Note that the actual gamma matrix has an addition factor of $\epsilon^2$ sitting in front)

In [10]:
gamma = np.zeros((d**2-1,d**2-1),dtype = np.cdouble)

for i in range(d**2-1):
    for k in range(d**2-1):
        gamma[i,k] = a_coeff[i].conj()*b_coeff[k] + cprime_coeff[i].conj()*aprime_coeff[k]+b_coeff[i].conj()*a_coeff[k]+aprime_coeff[i].conj()*cprime_coeff[k]


In [11]:
## Store the matrix in a CSV file

labels = []
for i in range(15):
    labels.append(str(i+1))
    
pd.set_option("display.precision", 2)

df = pd.DataFrame(gamma,columns = labels,index = labels) #creates a data frame
df.to_csv("gamma.csv")


In [12]:
print(df)

           1           2           3         4           5           6  \
1   0.0+0.0j  0.00+0.00j  0.00+0.00j  0.0+0.0j  0.00+0.00j  0.00+0.00j   
2   0.0+0.0j  0.00+0.00j  1.54+3.43j  0.0+0.0j  0.00+0.00j  0.01+0.05j   
3   0.0+0.0j  1.54-3.43j  0.00+0.00j  0.0+0.0j -0.18-0.01j  0.00+0.00j   
4   0.0+0.0j  0.00+0.00j  0.00+0.00j  0.0+0.0j  0.00+0.00j  0.00+0.00j   
5   0.0+0.0j  0.00+0.00j -0.18+0.01j  0.0+0.0j  0.00+0.00j  0.00+0.00j   
6   0.0+0.0j  0.01-0.05j  0.00+0.00j  0.0+0.0j  0.00+0.00j  0.00+0.00j   
7   0.0+0.0j  0.00+0.00j  0.18-0.01j  0.0+0.0j  0.00+0.00j  0.00+0.00j   
8   0.0+0.0j  0.00+0.00j  0.00+0.00j  0.0+0.0j  0.00+0.00j  0.00+0.00j   
9   0.0+0.0j  0.00+0.00j  0.00+0.00j  0.0+0.0j  0.00+0.00j  0.00+0.00j   
10  0.0+0.0j  0.00+0.00j  0.00+0.00j  0.0+0.0j  0.00+0.00j  0.00+0.00j   
11  0.0+0.0j  0.00+0.00j  0.00+0.00j  0.0+0.0j  0.00+0.00j  0.00+0.00j   
12  0.0+0.0j  0.00+0.00j  0.00+0.00j  0.0+0.0j  0.00+0.00j  0.00+0.00j   
13  0.0+0.0j  0.00+0.00j  0.00+0.00j  